# New methane lifetime treatment within FaIR

- all species emissions driven
- interactive temperatures
- CMIP6 model calibrations

We might expect historical lifetimes to be marginally off as it's possible model-generated temperatures will be too warm.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
import scipy.stats
from tqdm import tqdm

from fair21 import FAIR
from fair21.interface import initialise, fill
from fair21.io import read_properties

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
df = pd.read_csv("../data/calibration/4xCO2_cummins.csv")
models = df['model'].unique()

configs = []

seedgen = 1355763
for imodel, model in enumerate(models):
    for run in df.loc[df['model']==model, 'run']:
#        condition = (df['model']==model) & (df['run']==run)
        configs.append(f"{model}_{run}")
#         climate_response = ClimateResponse(
#             ocean_heat_capacity = df.loc[condition, 'C1':'C3'].values.squeeze(),
#             ocean_heat_transfer = df.loc[condition, 'kappa1':'kappa3'].values.squeeze(),
#             deep_ocean_efficacy = df.loc[condition, 'epsilon'].values[0],
#             gamma_autocorrelation = df.loc[condition, 'gamma'].values[0],
#             sigma_eta = df.loc[condition, 'sigma_eta'].values[0],
#             sigma_xi = df.loc[condition, 'sigma_xi'].values[0],
#             stochastic_run = True,
#             seed = seedgen
#         )
#         seedgen = seedgen+399
#         species_config = [species_config_from_default(species) for species in species_to_include]
        
#         # edit the methane characteristics
#         species_config[2].lifetime = 10.788405534387858
#         species_config[2].natural_emissions_adjustment = 19.019783117809567
#         species_config[2].soil_lifetime = 185
        
#         configs.append(Config(config_name, climate_response, species_config))

In [ ]:
species, properties = read_properties()

In [ ]:
f = FAIR(ch4_method='thornhill2021')
f.define_time(1750, 2500, 1)
f.define_configs(configs)
f.define_scenarios(scenarios)
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
seed = 1355763

fill(f.climate_configs['stochastic_run'], True)
fill(f.climate_configs['use_seed'], True)

for imodel, model in enumerate(models):
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        config = f"{model}_{run}"
        fill(f.climate_configs['ocean_heat_capacity'], df.loc[condition, 'C1':'C3'].values.squeeze(), config=config)
        fill(f.climate_configs['ocean_heat_transfer'], df.loc[condition, 'kappa1':'kappa3'].values.squeeze(), config=config)
        fill(f.climate_configs['deep_ocean_efficacy'], df.loc[condition, 'epsilon'].values[0], config=config)
        fill(f.climate_configs['gamma_autocorrelation'], df.loc[condition, 'gamma'].values[0], config=config)
        fill(f.climate_configs['sigma_eta'], df.loc[condition, 'sigma_eta'].values[0], config=config)
        fill(f.climate_configs['sigma_xi'], df.loc[condition, 'sigma_xi'].values[0], config=config)
        fill(f.climate_configs['seed'], seed, config=config)
        
        seed=seed+399

In [ ]:
f.fill_species_configs()

In [ ]:
fill(f.species_configs['unperturbed_lifetime'], 10.4198121, specie='CH4')
fill(f.species_configs['baseline_emissions'], 19.019783117809567, specie='CH4')
fill(f.species_configs['baseline_emissions'], 0.08602230754, specie='N2O')

In [ ]:
# this is slow, slower than running FaIR! We should move to a binary format and purge all the junk
f.fill_from_rcmip()

In [ ]:
initialise(f.temperature, 0)
initialise(f.forcing, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.concentration, f.species_configs['baseline_concentration'])

In [ ]:
f.species_configs

In [ ]:
f.run()

In [ ]:
df_conc = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
conc_ch4 = {}
for scenario in scenarios:
    conc_ch4[scenario] = df_conc.loc[
        (df_conc['Scenario']==scenario) & (df_conc['Variable'].str.endswith("|CH4")) & 
        (df_conc['Region']=='World'), '1750':'2500'
    ].interpolate(axis=1).values.squeeze()

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16,9))
for iscen, scenario in enumerate(scenarios):
    ax[iscen//4, iscen%4].plot(f.timebounds, f.concentration.loc[dict(scenario=scenario, specie='CH4')]);
    ax[iscen//4, iscen%4].plot(f.timebounds, conc_ch4[scenario][:], color='k', label='MAGICC CMIP6')
    ax[iscen//4, iscen%4].set_title(scenario)
ax[iscen//4, iscen%4].legend();

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16,9))
for i in range(8):
    ax[i//4, i%4].plot(f.alpha_lifetime[:,i,:,3] * 10.4198121);
    ax[i//4, i%4].set_title(scenarios[i])

In [ ]:
fig, ax = pl.subplots(2, 4, figsize=(16, 10))

for i in range(8):
    ax[i//4,i%4].fill_between(
        f.timebounds, 
        np.min(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), axis=1), 
        np.max(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        f.timebounds, 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 5, axis=1), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 95, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].fill_between(
        f.timebounds, 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 16, axis=1), 
        np.percentile(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), 84, axis=1),
        color='#000000',
        alpha=0.2,
    )
    ax[i//4,i%4].plot(
        f.timebounds, 
        np.median(f.temperature[:, i, :, 0]-f.temperature[100:151, i, :, 0].mean(axis=0), axis=1), 
        color='#000000',
    )
    ax[i//4, i%4].plot(2021, 1.2, 'bx')
    ax[i//4,i%4].set_xlim(1850,2100)
    ax[i//4,i%4].set_ylim(-1, 10)
    ax[i//4,i%4].axhline(0, color='k', ls=":", lw=0.5)
    ax[i//4,i%4].set_title(scenarios[i])
pl.suptitle('Temperature anomaly')

In [ ]:
f.concentration[269, 3, :, 3]